In [1]:
import os
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from utils import tools
from modules.vae_base_module import VAEBaseModule
from models.vae_simple import SimpleVAE
from datasets.novelty_mnist import NoveltyMNISTDataModule

%matplotlib inline
plt.style.use('seaborn')


In [2]:
default_path = os.path.join(os.getcwd(), '../configs/vae/vae_simple_emnist.yaml')
config = tools.config_from_command_line(default_path)

Experimental parameters
------
{'data-parameters': {'batch_size': 128,
                     'download': True,
                     'root_data_path': '/home/sleipnir/Documents/Datasets/NoveltyMNIST',
                     'train_fraction': 0.9},
 'experiment-parameters': {'datamodule': 'NoveltyMNISTDataModule',
                           'log_dir': 'logs',
                           'name': 'VAESimple'},
 'module-parameters': {'latent_dims': 10, 'learning_rate': 0.001}}


In [4]:
datamodule = NoveltyMNISTDataModule(**config['data-parameters'])
datamodule.setup('test')

model = SimpleVAE(datamodule.shape, **config['module-parameters'])

module = VAEBaseModule(model, **config['module-parameters'])

checkpoint = torch.load('../logs/VAESimple/version_0/checkpoints/epoch=29-step=10109.ckpt')
module.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
test_novelty_scores = []
test_novelty_labels = []

module.model.eval()
with torch.no_grad():
    for batch_nb, batch_tuple in enumerate(datamodule.test_dataloader()):
        result = module.test_step();
    # enumerate dataloader
    # write test step for module
    # return novelty scores
